In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=f9c2ba6881731ccf45689ab7f38377348e39fa55a2264758704db87f8ea35c5f
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession 

In [4]:
spark=SparkSession.builder.appName('Dataframe Examples').getOrCreate()

In [5]:
sc=spark.sparkContext

In [6]:
rdd=sc.parallelize([3,4,5,6],2)

In [8]:
rdd2=sc.textFile('/content/5000 Sales Records.csv')

In [9]:
rdd2.getNumPartitions()

2

In [33]:
rdd2.take(2)

['Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit',
 'Central America and the Caribbean,Antigua and Barbuda ,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72']

In [11]:
rdd3=rdd2.map(lambda x: x.split(','))

In [12]:
rdd3.take(2)

[['Region',
  'Country',
  'Item Type',
  'Sales Channel',
  'Order Priority',
  'Order Date',
  'Order ID',
  'Ship Date',
  'Units Sold',
  'Unit Price',
  'Unit Cost',
  'Total Revenue',
  'Total Cost',
  'Total Profit'],
 ['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  '12/20/2013',
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72']]

In [13]:
header=rdd3.first()

In [14]:
rdd4=rdd3.filter(lambda x:x!=header)

In [15]:
rdd4.take(2)

[['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  '12/20/2013',
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72'],
 ['Central America and the Caribbean',
  'Panama',
  'Snacks',
  'Offline',
  'C',
  '7/5/2010',
  '301644504',
  '7/26/2010',
  '2167',
  '152.58',
  '97.44',
  '330640.86',
  '211152.48',
  '119488.38']]

In [16]:
rdd4.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [17]:
rdd5=rdd4.cache()

In [18]:
rdd5.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [19]:
# Display the count of number of countries
sol1=rdd5.map(lambda x : x[1]).distinct().count()

In [20]:
sol1

185

In [21]:
# Display the number of units sold in each region
#sol2=rdd5.map(lambda x: (x[0],int(x[8]))).groupByKey().mapValues(lambda x: sum(x)) # Using mapValues
sol2=rdd5.map(lambda x: (x[0],int(x[8]))).reduceByKey(lambda x,y:x+y) # Using reduceByKey

In [22]:
sol2.collect()

[('Asia', 3620036),
 ('Middle East and North Africa', 3013431),
 ('Australia and Oceania', 2111786),
 ('Central America and the Caribbean', 2698776),
 ('Europe', 6582322),
 ('Sub-Saharan Africa', 6642380),
 ('North America', 484760)]

In [23]:

from datetime import datetime

In [24]:
# Display the 10 most recent Sales

sol3=rdd5.map(lambda x: (x[5] , datetime.strptime(x[5],'%m/%d/%Y')))
sol3.sortBy(lambda x:x[1],ascending= False).take(10)

[('7/28/2017', datetime.datetime(2017, 7, 28, 0, 0)),
 ('7/26/2017', datetime.datetime(2017, 7, 26, 0, 0)),
 ('7/26/2017', datetime.datetime(2017, 7, 26, 0, 0)),
 ('7/26/2017', datetime.datetime(2017, 7, 26, 0, 0)),
 ('7/24/2017', datetime.datetime(2017, 7, 24, 0, 0)),
 ('7/24/2017', datetime.datetime(2017, 7, 24, 0, 0)),
 ('7/24/2017', datetime.datetime(2017, 7, 24, 0, 0)),
 ('7/23/2017', datetime.datetime(2017, 7, 23, 0, 0)),
 ('7/22/2017', datetime.datetime(2017, 7, 22, 0, 0)),
 ('7/21/2017', datetime.datetime(2017, 7, 21, 0, 0))]

In [25]:

# Display the products with atleast 2 occurance of 'a'
sol4=rdd5.map(lambda x: (x[2],x[2].count('a'))).filter(lambda x: x[1]>=2)


In [26]:
sol4.take(2)

[('Personal Care', 2), ('Personal Care', 2)]

In [27]:
# Display the country in each region with highest units sold
sol5=rdd5.map(lambda x: (x[0],x[1],int(x[8]))).groupBy(lambda x:(x[0],x[1])).mapValues(lambda x:max(x))

In [28]:
sol5.take(2)

[(('Central America and the Caribbean', 'Panama'),
  ('Central America and the Caribbean', 'Panama', 9976)),
 (('Europe', 'Czech Republic'), ('Europe', 'Czech Republic', 9759))]

In [31]:
sol5a=sol5.map(lambda x: x[1])

In [32]:
sol5a.take(2)

[('Central America and the Caribbean', 'Panama', 9976),
 ('Europe', 'Czech Republic', 9759)]

In [50]:
# Display the unit price and unit cost of each item in ascending order
sol6=rdd5.map(lambda x: (x[2],float(x[9]),float(x[10]))).sortBy(lambda x:x[0],ascending=True).distinct()

In [51]:
sol6.take(2)

[('Beverages', 47.45, 31.79), ('Fruits', 9.33, 6.92)]

In [65]:
# Display the number of Sales Yearwise
sol7=rdd5.map(lambda x: ((x[5][-4:]),int(x[8]))).groupByKey().mapValues(lambda x: sum(x))

In [66]:
sol7.take(2)

[('2013', 3358584), ('2010', 3130137)]

In [69]:
# Dispay number of orders for each item
sol8=rdd5.map(lambda x: (x[2],1)).groupByKey().mapValues(lambda x:sum(x))

In [70]:
sol8.take(2)

[('Baby Food', 445), ('Snacks', 398)]

In [82]:
# Display the country with highest sale
sol9=rdd5.map(lambda x:(x[1],int(x[8]))).groupByKey().mapValues(lambda x:sum(x)).max()

In [84]:
sol9

('Zimbabwe', 93906)

In [ ]:
sol8.save